In [4]:
import os

IMG_SIZE = (224, 224)
BATCH_SIZE = 16
SEED = 42

DATASET_DIR = "dataset/fruits-360"
TRAIN_DIR = os.path.join(DATASET_DIR, "Training")
TEST_DIR  = os.path.join(DATASET_DIR, "Test")

In [5]:
print("Training exists:", os.path.exists(TRAIN_DIR))
print("Test exists:", os.path.exists(TEST_DIR))
print("Number of training classes:", len(os.listdir(TRAIN_DIR)))

Training exists: True
Test exists: True
Number of training classes: 245


In [6]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

NUM_CLASSES = len(train_ds.class_names)
print("Number of classes:", NUM_CLASSES)


Found 128252 files belonging to 245 classes.


2026-01-26 20:10:03.131601: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2026-01-26 20:10:03.131750: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-01-26 20:10:03.131756: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2026-01-26 20:10:03.131786: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-26 20:10:03.131795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 42749 files belonging to 245 classes.
Number of classes: 245


In [7]:
from tensorflow.keras import layers
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

def train_preprocess(x, y):
    x = data_augmentation(x)
    x = preprocess_input(x)
    return x, y

def val_preprocess(x, y):
    x = preprocess_input(x)
    return x, y

train_ds = train_ds.map(train_preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.map(val_preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models

base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # Transfer learning

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 245)            │        62,965 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,648,885 (10.10 MB)

 Trainable params: 390,901 (1.49 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10


2026-01-26 20:12:49.761475: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


8016/8016 ━━━━━━━━━━━━━━━━━━━━ 775s 96ms/step - accuracy: 0.6589 - loss: 1.4873 - val_accuracy: 0.9243 - val_loss: 0.3419
Epoch 2/10
8016/8016 ━━━━━━━━━━━━━━━━━━━━ 562s 70ms/step - accuracy: 0.9283 - loss: 0.2687 - val_accuracy: 0.9568 - val_loss: 0.1660
Epoch 3/10
8016/8016 ━━━━━━━━━━━━━━━━━━━━ 1182s 147ms/step - accuracy: 0.9617 - loss: 0.1384 - val_accuracy: 0.9723 - val_loss: 0.1065
Epoch 4/10
8016/8016 ━━━━━━━━━━━━━━━━━━━━ 562s 70ms/step - accuracy: 0.9746 - loss: 0.0911 - val_accuracy: 0.9789 - val_loss: 0.0815
Epoch 5/10
8016/8016 ━━━━━━━━━━━━━━━━━━━━ 975s 122ms/step - accuracy: 0.9803 - loss: 0.0688 - val_accuracy: 0.9785 - val_loss: 0.0724
Epoch 6/10
8016/8016 ━━━━━━━━━━━━━━━━━━━━ 5287s 660ms/step - accuracy: 0.9840 - loss: 0.0559 - val_accuracy: 0.9794 - val_loss: 0.0660
Epoch 7/10
8016/8016 ━━━━━━━━━━━━━━━━━━━━ 714s 89ms/step - accuracy: 0.9862 - loss: 0.0465 - val_accuracy: 0.9843 - val_loss: 0.0541
Epoch 8/10
8016/8016 ━━━━━━━━━━━━━━━━━━━━ 558s 70ms/step - accuracy: 0.9879

In [1]:
# Unfreeze the base model
base_model.trainable = True

# Freeze lower layers, fine-tune higher layers only
for layer in base_model.layers[:100]:
    layer.trainable = False

NameError: name 'base_model' is not defined

In [ ]:
#🔹 Recompile with Lower Learning Rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
#Lower LR is critical during fine-tuning.

IndentationError: unexpected indent (4261499620.py, line 2)

In [ ]:
#Fine-Tune
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# Confusion Matrix & Class-Wise Accuracy

# Get Predictions
import numpy as np

y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=train_ds.class_names
)

disp.plot(cmap="Blues", xticks_rotation=90)
plt.show()

In [ ]:
# Class-Wise Accuracy
import pandas as pd

class_acc = cm.diagonal() / cm.sum(axis=1)

df = pd.DataFrame({
    "Class": train_ds.class_names,
    "Accuracy": class_acc
})

print(df)

# Robustness Testing (Noise Analysis)

# Gaussian Noise Function
def add_gaussian_noise(images, std=0.05):
    noise = tf.random.normal(tf.shape(images), mean=0.0, stddev=std)
    images = images + noise
    return tf.clip_by_value(images, 0.0, 255.0)

In [ ]:
# Noisy Validation Dataset
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def noisy_preprocess(x, y):
    x = add_gaussian_noise(x)
    x = preprocess_input(x)
    return x, y

noisy_val_ds = val_ds.map(noisy_preprocess)

In [ ]:
# Evaluation Under Noise
model.evaluate(noisy_val_ds)
# Report accuracy drop under noise → strong research contribution.

In [ ]:
# Saving model
model.save("fruits_mobilenet.h5")

# Convert to TensorFlow Lite (FP32)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("fruits_fp32.tflite", "wb") as f:
    f.write(tflite_model)